# Notebook to train and evaluate Yolo Mlt (Yolov5 small for object detection and image classification)

As of today (December 2nd 2022), the goal is to detect [speed signs, road work signs/objects, the horizon of the road] and to classify the road condition [dry/snowy/wet].

### Imports

In [7]:
import glob
import os

In [29]:
pwd

'/home/selim/Desktop/yolov5_multitask'

## Training

### Train the detection task on esmart_wip dataset

Let's start by training the detection task with all layers activated (besides the classification head just as a security).

In [5]:
weights = 'runs/train-mlt/exp137/weights/best.pt'
!python multitasks/train.py --epochs 50 --img 512 --weights {weights} --data ../datasets/esmart_wip/data.yaml --batch-size 32 --only_det --freeze 25

wandb: Currently logged in as: selimgilon (esmart). Use `wandb login --relogin` to force relogin
multitasks/train: weights=runs/train-mlt/exp137/weights/best.pt, cfg=models/yolov5s_mlt.yaml, data=../datasets/esmart_wip/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=150, batch_size=32, imgsz=512, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train-mlt, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[25], freeze_till=[0], freeze_all_but=[], only_cls=False, only_det=True, save_period=-1, seed=0, local_rank=-1, mlt=[0], entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 41 commits. Use `git pull ultralytics master` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5

In [8]:
# get the last folder of the folder runs/train-mlt/
list_of_files = glob.glob('runs/train-mlt/*')
latest_exp_path = max(list_of_files, key=os.path.getmtime)
print(latest_exp_path)
last_weights = f"{latest_exp_path}/weights/last.pt"
best_weights = f"{latest_exp_path}/weights/best.pt"

runs/train-mlt/exp144


### Train the classification task on esmart_context

Train only the classification using the weights form the previous run and freezing every layer besides the classification head (and optionally the layer before #8)

In [25]:
!python multitasks/train.py --epochs 50 --img 512 --weights runs/train-mlt/exp112/weights/last.pt --data ../datasets/esmart_context/data.yaml  --batch-size 32 --only_cls --freeze_all_but 8 25 --cut_img 0.5
#!python multitasks/train.py --epochs 50 --img 512 --weights {best_weights} --data ../datasets/esmart_context/data.yaml  --batch-size 32 --only_cls --freeze_all_but 25 --cut_img 0.5

wandb: Currently logged in as: selimgilon (esmart). Use `wandb login --relogin` to force relogin
multitasks/train: weights=runs/train-mlt/exp112/weights/last.pt, cfg=models/yolov5s_mlt.yaml, data=../datasets/esmart_context/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=32, imgsz=512, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train-mlt, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, cut_img=0.5, patience=100, freeze=[], freeze_all=False, freeze_till=[0], freeze_all_but=[8, 25], only_cls=True, only_det=False, save_period=-1, seed=0, local_rank=-1, mlt=[0], entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 42 commits. Use `git pull ultralytics master` or `git clone https://github.com/

In [ ]:
# get the last folder of the folder runs/train-mlt/
list_of_files = glob.glob('runs/train-mlt/*')
latest_exp_path = max(list_of_files, key=os.path.getmtime)
print(latest_exp_path)
last_weights = f"{latest_exp_path}/weights/last.pt"
best_weights = f"{latest_exp_path}/weights/best.pt"

### Train again the detection task on esmart_wip

Train only the detection using the weights form the previous run and freezing every layer in the backbone and classification head.

In [27]:
!python multitasks/train.py --epochs 10 --img 512 --weights runs/train-mlt/exp160/weights/last.pt --data ../datasets/esmart_wip/data.yaml --batch-size 32 --only_det --freeze 0 1 2 3 4 5 6 7 8 25

wandb: Currently logged in as: selimgilon (esmart). Use `wandb login --relogin` to force relogin
multitasks/train: weights=runs/train-mlt/exp160/weights/last.pt, cfg=models/yolov5s_mlt.yaml, data=../datasets/esmart_wip/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=32, imgsz=512, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train-mlt, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, cut_img=0.0, patience=100, freeze=[0, 1, 2, 3, 4, 5, 6, 7, 8, 25], freeze_all=False, freeze_till=[0], freeze_all_but=[], only_cls=False, only_det=True, save_period=-1, seed=0, local_rank=-1, mlt=[0], entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (13/13), done.
remote: Co

In [16]:
list_of_files = glob.glob('runs/train-mlt/*')
latest_exp_path = max(list_of_files, key=os.path.getmtime)
print(latest_exp_path)
last_weights = f"{latest_exp_path}/weights/last.pt"
best_weights = f"{latest_exp_path}/weights/best.pt"

runs/train-mlt/exp146


# Evaluate

In [46]:
weights_path = 'runs/train-mlt/exp162/weights/best.pt'

### The road conditions classification on esmart_context

Note that the display of the logger works well in the terminal but sometimes doesn't show the title of the metrics when running from a notebook.
Here is the list of the reported metrics (in order):

*Class, Images, Instances, cls_P, cls_R, cls_fpr, cls_f1, cls_acc*

In [49]:
!python multitasks/val.py --img 512 --weights {weights_path} --data ../datasets/esmart_context/data.yaml  --batch-size 32 --only_cls_eval

multitasks/val: data=../datasets/esmart_context/data.yaml, weights=['runs/train-mlt/exp162/weights/best.pt'], batch_size=32, imgsz=512, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=multitasks/runs/val-mlt, name=exp, exist_ok=False, half=False, dnn=False, only_det_eval=False, only_cls_eval=True
YOLOv5 🚀 v6.2-273-g2253fbb5 Python-3.8.10 torch-1.12.0+cu102 CUDA:0 (NVIDIA GeForce RTX 2080 SUPER, 7960MiB)

Fusing layers... 
YOLOv5s_mlt summary: 163 layers, 7697578 parameters, 0 gradients, 16.4 GFLOPs
val: Scanning '/home/selim/Desktop/esmart-ai-datasets/data/esmart_context/val.ca
                 Class     Images  Instances      cls_P      cls_R    cls_fpr   
                   all       8971       8971      0.831      0.799      0.169      0.787      0.784
                   dry       8971       3745      0.972      0.717     0.0279    


### The detections on esmart_wip

*Class, Images, Instances, P, R, mAP50, mAP50-95*

In [50]:
!python multitasks/val.py --img 512 --weights {weights_path} --data ../datasets/esmart_wip/data.yaml --batch-size 32 --only_det_eval

multitasks/val: data=../datasets/esmart_wip/data.yaml, weights=['runs/train-mlt/exp162/weights/best.pt'], batch_size=32, imgsz=512, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=multitasks/runs/val-mlt, name=exp, exist_ok=False, half=False, dnn=False, only_det_eval=True, only_cls_eval=False
YOLOv5 🚀 v6.2-273-g2253fbb5 Python-3.8.10 torch-1.12.0+cu102 CUDA:0 (NVIDIA GeForce RTX 2080 SUPER, 7960MiB)

Fusing layers... 
YOLOv5s_mlt summary: 163 layers, 7697578 parameters, 0 gradients, 16.4 GFLOPs
val: Scanning '/home/selim/Desktop/esmart-ai-datasets/data/esmart_wip/val.cache'
val: WARNING ⚠️ /home/selim/Desktop/esmart-ai-datasets/data/esmart_wip/run_7_12704.jpg: 8 duplicate labels removed
                 Class     Images  Instances          P          R      mAP50   
                   all        516       1311      0.375      0.343    

## Export the model (work in progress)

In [ ]:
!python export.py --weights runs/train-mlt/exp162/weights/last.pt --include onnx